In [2]:
!pip install emoji contractions
!pip install lightgbm

   ---------------------------------------- 0.0/590.6 kB ? eta -:--:--
   ---------------------------------------- 590.6/590.6 kB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   -------------- ------------------------- 0.5/1.5 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 4.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import joblib
import re
import emoji
import contractions
import seaborn as sns
import matplotlib.pyplot as plt
import random
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputRegressor
from scipy.sparse import hstack
from scipy.stats import pearsonr
from collections import defaultdict

from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.multiclass import OneVsRestClassifier
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier

# Data Preparation

In [12]:
# Load datasets
train = pd.read_csv('train.csv', delimiter='\t')
test = pd.read_csv('test.csv', delimiter='\t')
dev = pd.read_csv('dev.csv', delimiter='\t')

combined_df = pd.concat([train, dev, test], ignore_index=True)

# Atur parameter test_size di Streamlit
train_df, test_df = train_test_split(combined_df, test_size=0.2, random_state=42)

print(f"Training set: {train_df.shape}")
print(f"Test set: {test_df.shape}")

Training set: (8872, 6)
Test set: (2218, 6)


## Data Cleaning

In [13]:
# Text preprocessing functions
def convert_emojis(text):
    text = emoji.demojize(text, delimiters=(" ", " "))
    text = re.sub(r':([a-zA-Z_]+):', r'\1', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def clean_text(text):
    # Lowercase
    text = text.lower()
    # expand contractions
    text = contractions.fix(text)
    # convert emojis
    text = convert_emojis(text)
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    # Remove user mentions and hashtags
    # text = re.sub(r'\@\w+|\#','', text)
    text = re.sub(r'@\w+', '', text)
    # Remove special characters and numbers (except punctuation)
    text = re.sub(r"[^a-zA-Z\s.,!?']", '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply text cleaning
train_df["clean_text"] = train_df["Tweet"].apply(clean_text)
test_df["clean_text"] = test_df["Tweet"].apply(clean_text)

# Define emotion columns
emotion_cols = ["joy", "sadness", "anger", "fear"]

In [15]:
# Check for missing values in emotion columns
print("Missing values in emotion columns:")
for df, name in [(train_df, "train"), (test_df, "test")]:
    print(f"\n{name} dataset:")
    for col in emotion_cols:
        missing = df[col].isna().sum()
        total = len(df)
        print(f"{col}: {missing} missing values ({missing/total*100:.1f}%)")

# Fill missing values with 0 (indicating absence of that emotion)
for df in [train_df, test_df]:
    for col in emotion_cols:
        df[col] = df[col].fillna(0.0)

Missing values in emotion columns:

train dataset:
joy: 0 missing values (0.0%)
sadness: 0 missing values (0.0%)
anger: 0 missing values (0.0%)
fear: 0 missing values (0.0%)

test dataset:
joy: 0 missing values (0.0%)
sadness: 0 missing values (0.0%)
anger: 0 missing values (0.0%)
fear: 0 missing values (0.0%)


## Lexicons

In [10]:
# Load EmoLex features
def load_lex(filepath):
    lexicon = defaultdict(dict)
    with open(filepath, 'r') as file:
        for line in file:
            word, emotion, value = line.strip().split('\t')
            if int(value) == 1:
                lexicon[word][emotion] = 1
    return lexicon

nrc_lexicon = load_lex("/kaggle/input/nrc-lexicons/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt")

def extract_lex(text, lexicon):
    emotions = ['anger', 'anticipation', 'disgust', 'fear', 'joy',
              'sadness', 'surprise', 'trust', 'positive', 'negative']
    counts = dict.fromkeys(emotions, 0)

    for word in text.split():
        if word in lexicon:
            for emo in lexicon[word]:
                counts[emo] += 1
    return [counts[emo] for emo in emotions]

# Extract lexicon features
train_df['lexicons'] = train_df['clean_text'].apply(lambda x: extract_lex(x, nrc_lexicon))
test_df['lexicons'] = test_df['clean_text'].apply(lambda x: extract_lex(x, nrc_lexicon))

train_lex = np.array(train_df['lexicons'].tolist())
test_lex = np.array(test_df['lexicons'].tolist())

FileNotFoundError: [Errno 2] No such file or directory: 'NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'

In [ ]:
# Load VAD Lexicons
def load_nrc_vad(filepath):
    vad_lex = {}
    with open(filepath, 'r', encoding='utf-8') as f:
        next(f)  # skip header
        for line in f:
            word, val, aro, dom = line.strip().split('\t')
            vad_lex[word] = {
                'valence': float(val),
                'arousal': float(aro),
                'dominance': float(dom)
            }
    return vad_lex

nrc_vad_lexicon = load_nrc_vad("NRC-VAD-Lexicon-v2.1.txt")

def extract_vad(text, lexicon):
    valence = []
    arousal = []
    dominance = []

    for word in text.split():
        if word in lexicon:
            valence.append(lexicon[word]['valence'])
            arousal.append(lexicon[word]['arousal'])
            dominance.append(lexicon[word]['dominance'])

    # If no word matched, return zeros
    if not valence:
        return [0.0, 0.0, 0.0]

    # Otherwise, return means
    return [
        np.mean(valence),
        np.mean(arousal),
        np.mean(dominance)
    ]

# Extract lexicon features
train_df['vad'] = train_df['clean_text'].apply(lambda x: extract_vad(x, nrc_vad_lexicon))
test_df['vad'] = test_df['clean_text'].apply(lambda x: extract_vad(x, nrc_vad_lexicon))

train_vad = np.array(train_df['vad'].tolist())
test_vad = np.array(test_df['vad'].tolist())

In [ ]:
# Load HashEmo Lexicons
def load_nrc_hash_emo(filepath):
    lexicon = defaultdict(dict)
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            emotion, word, score = line.strip().split('\t')
            lexicon[word][emotion] = float(score)
    return lexicon

hash_emo_lex = load_nrc_hash_emo('NRC-Hashtag-Emotion-Lexicon-v0.2.txt')

def extract_hash_emo(text, lexicon):
    emotions = ['anger', 'anticipation', 'disgust', 'fear', 'joy',
                'sadness', 'surprise', 'trust']
    scores = {emo: [] for emo in emotions}

    for word in text.split():
        if word in lexicon:
            for emo, value in lexicon[word].items():
                scores[emo].append(value)

    return [np.mean(scores[emo]) if scores[emo] else 0.0 for emo in emotions]

train_df['hash'] = train_df['clean_text'].apply(lambda x: extract_hash_emo(x, hash_emo_lex))
test_df['hash'] = test_df['clean_text'].apply(lambda x: extract_hash_emo(x, hash_emo_lex))

train_hash = np.array(train_df['hash'].tolist())
test_hash = np.array(test_df['hash'].tolist())

In [9]:
scaler_hash = StandardScaler()
train_hash = scaler_hash.fit_transform(train_hash)
test_hash = scaler_hash.transform(test_hash)

scaler_lex = StandardScaler()
train_lex = scaler_lex.fit_transform(train_lex)
test_lex = scaler_lex.transform(test_lex)

scaler_vad = StandardScaler()
train_vad = scaler_vad.fit_transform(train_vad)
test_vad = scaler_vad.transform(test_vad)

In [10]:
# NRC Hash-Emo + EmoLex + VAD
train_combined = np.concatenate([train_vad, train_lex, train_hash], axis=1)
test_combined = np.concatenate([test_vad, test_lex, test_hash], axis=1)

## Extract Embeddings

In [11]:
# TF-IDF Vectorization
tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    stop_words='english'
)

# Fit on training set and transform all sets
train_tfidf = tfidf.fit_transform(train_df['clean_text'])
test_tfidf = tfidf.transform(test_df['clean_text'])

## Training Preparation

In [12]:
# TF-IDF is sparse, so we use hstack (sparse-safe)
X_train = hstack([train_tfidf, train_combined])
X_test = hstack([test_tfidf, test_combined])

In [13]:
# y_train_cls = (train_df[emotion_cols] > 0).astype(int).values
# y_test_cls  = (test_df[emotion_cols] > 0).astype(int).values

y_train_cls = train_df[emotion_cols].values.argmax(axis=1)
y_test_cls = test_df[emotion_cols].values.argmax(axis=1)

label_map = {0: 'joy', 1: 'sadness', 2: 'anger', 3: 'fear'}

y_train_reg = train_df[emotion_cols].values
y_test_reg = test_df[emotion_cols].values

# Regression Model

Optional Streamlit model toggle to choose model user prefers (default ensemble because it's best performing)

## Ridge Regression

In [14]:
# Initialize model
ridge = Ridge(alpha=1.0, solver='lsqr', random_state=42)
ridge_reg = MultiOutputRegressor(ridge)

ridge_reg.fit(X_train, y_train_reg)

y_pred_ridge = ridge_reg.predict(X_test)

for i, emotion in enumerate(emotion_cols):
    print(f"\nEmotion: {emotion}")
    print(f"  MAE: {mean_absolute_error(y_test_reg[:, i], y_pred_ridge[:, i]):.4f}")
    print(f"  MSE: {mean_squared_error(y_test_reg[:, i], y_pred_ridge[:, i]):.4f}")
    print(f"  R^2: {r2_score(y_test_reg[:, i], y_pred_ridge[:, i]):.4f}")
    corr, _ = pearsonr(y_test_reg[:, i], y_pred_ridge[:, i])
    print(f"  Pearson: {corr:.4f}")


Emotion: joy
  MAE: 0.1078
  MSE: 0.0225
  R^2: 0.6538
  Pearson: 0.8087

Emotion: sadness
  MAE: 0.1198
  MSE: 0.0289
  R^2: 0.5086
  Pearson: 0.7143

Emotion: anger
  MAE: 0.1184
  MSE: 0.0280
  R^2: 0.5291
  Pearson: 0.7277

Emotion: fear
  MAE: 0.1274
  MSE: 0.0303
  R^2: 0.5404
  Pearson: 0.7358


## LightGBM Regressor

In [ ]:
lgbm_base = LGBMRegressor(
    num_leaves=20,
    n_estimators=500,
    learning_rate=0.1,
    reg_alpha=0.1,
    min_child_samples=3,
    colsample_bytree=0.3,
    random_state=42,
    n_jobs=-1
)

lgbm_reg = MultiOutputRegressor(lgbm_base)

lgbm_reg.fit(X_train, y_train_reg)

y_pred_lgb = lgbm_reg.predict(X_test)

for i, emotion in enumerate(emotion_cols):
    print(f"\nEmotion: {emotion}")
    print(f"  MAE: {mean_absolute_error(y_test_reg[:, i], y_pred_lgb[:, i]):.4f}")
    print(f"  MSE: {mean_squared_error(y_test_reg[:, i], y_pred_lgb[:, i]):.4f}")
    print(f"  R^2: {r2_score(y_test_reg[:, i], y_pred_lgb[:, i]):.4f}")
    corr, _ = pearsonr(y_test_reg[:, i], y_pred_lgb[:, i])
    print(f"  Pearson: {corr:.4f}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.191765 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 28074
[LightGBM] [Info] Number of data points in the train set: 8872, number of used features: 4885
[LightGBM] [Info] Start training from score 0.136371
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.191479 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 28074
[LightGBM] [Info] Number of data points in the train set: 8872, number of used features: 4885
[LightGBM] [Info] Start training from score 0.132116
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.211279 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Tota

/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:1218: UserWarning: Converting data to scipy sparse matrix.
  _log_warning("Converting data to scipy sparse matrix.")


## Ensemble (Ridge & LightGBM)

In [16]:
from sklearn.base import BaseEstimator, RegressorMixin

class EnsembleRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, model1, model2, alpha=0.3):
        self.model1 = model1
        self.model2 = model2
        self.alpha = alpha

    def fit(self, X, y):
        self.model1.fit(X, y)
        self.model2.fit(X, y)
        return self

    def predict(self, X):
        pred1 = self.model1.predict(X)
        pred2 = self.model2.predict(X)
        return self.alpha * pred1 + (1 - self.alpha) * pred2

# Optional to set alpha in streamlit (default alpha = 0.3)
alpha = 0.3
ensemble_reg = EnsembleRegressor(model1=ridge_reg, model2=lgbm_reg, alpha=alpha)
ensemble_reg.fit(X_train, y_train_reg)
y_pred_ensemble = ensemble_reg.predict(X_test)

for i, emotion in enumerate(emotion_cols):
    print(f"\nEmotion: {emotion}")
    print(f"  MAE: {mean_absolute_error(y_test_reg[:, i], y_pred_ensemble[:, i]):.4f}")
    print(f"  MSE: {mean_squared_error(y_test_reg[:, i], y_pred_ensemble[:, i]):.4f}")
    print(f"  R^2: {r2_score(y_test_reg[:, i], y_pred_ensemble[:, i]):.4f}")
    corr, _ = pearsonr(y_test_reg[:, i], y_pred_ensemble[:, i])
    print(f"  Pearson: {corr:.4f}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.229930 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 28074
[LightGBM] [Info] Number of data points in the train set: 8872, number of used features: 4885
[LightGBM] [Info] Start training from score 0.136371
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.199843 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 28074
[LightGBM] [Info] Number of data points in the train set: 8872, number of used features: 4885
[LightGBM] [Info] Start training from score 0.132116
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.206862 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Tota

/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:1218: UserWarning: Converting data to scipy sparse matrix.
  _log_warning("Converting data to scipy sparse matrix.")



Emotion: joy
  MAE: 0.0875
  MSE: 0.0195
  R^2: 0.7005
  Pearson: 0.8373

Emotion: sadness
  MAE: 0.1022
  MSE: 0.0257
  R^2: 0.5634
  Pearson: 0.7506

Emotion: anger
  MAE: 0.1018
  MSE: 0.0251
  R^2: 0.5773
  Pearson: 0.7598

Emotion: fear
  MAE: 0.1143
  MSE: 0.0275
  R^2: 0.5832
  Pearson: 0.7639


# Classification Model

4 Optional models that can be toggled in Streamlit. Default Ensemble LR + LGBM because it got the best results.

## Logistic Regression

In [17]:
logR = LogisticRegression(max_iter=50, solver='newton-cg', random_state=42)
logR.fit(X_train, y_train_cls)

y_pred_logR = logR.predict(X_test)

print("Accuracy:", accuracy_score(y_test_cls, y_pred_logR))
print(classification_report(y_test_cls, y_pred_logR, target_names=emotion_cols))

Accuracy: 0.78809738503156
              precision    recall  f1-score   support

         joy       0.82      0.84      0.83       615
     sadness       0.78      0.73      0.75       475
       anger       0.81      0.77      0.79       526
        fear       0.75      0.80      0.78       602

    accuracy                           0.79      2218
   macro avg       0.79      0.78      0.79      2218
weighted avg       0.79      0.79      0.79      2218



## LightGBM Classifier

In [18]:
lgbm_clf = LGBMClassifier(
    num_leaves=20,
    n_estimators=500,
    learning_rate=0.1,
    reg_alpha=0.1,
    min_child_samples=3,
    colsample_bytree=0.5,
    random_state=42,
    n_jobs=-1
)

lgbm_clf.fit(X_train, y_train_cls)
y_pred_lgb_clf = lgbm_clf.predict(X_test)


print("Accuracy:", accuracy_score(y_test_cls, y_pred_lgb_clf))
print(classification_report(y_test_cls, y_pred_lgb_clf, target_names=emotion_cols))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.230088 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 28074
[LightGBM] [Info] Number of data points in the train set: 8872, number of used features: 4885
[LightGBM] [Info] Start training from score -1.326784
[LightGBM] [Info] Start training from score -1.548442
[LightGBM] [Info] Start training from score -1.439535
[LightGBM] [Info] Start training from score -1.255076
Accuracy: 0.8183047790802525
              precision    recall  f1-score   support

         joy       0.87      0.87      0.87       615
     sadness       0.81      0.78      0.79       475
       anger       0.85      0.79      0.82       526
        fear       0.76      0.82      0.79       602

    accuracy                           0.82      2218
   macro avg       0.82      0.82      0.82      2218
weighted avg       0

/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:1218: UserWarning: Converting data to scipy sparse matrix.
  _log_warning("Converting data to scipy sparse matrix.")


## Support Vector Machines (SVM)

In [19]:
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier

svm_model = SVC(kernel='linear', C=1.0, probability=True, max_iter=10000, random_state=42)
svm_model.fit(X_train, y_train_cls)
y_pred_svm = svm_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test_cls, y_pred_svm))
print(classification_report(y_test_cls, y_pred_svm, target_names=emotion_cols))

/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy: 0.8083859332732192
              precision    recall  f1-score   support

         joy       0.84      0.85      0.85       615
     sadness       0.78      0.78      0.78       475
       anger       0.85      0.77      0.81       526
        fear       0.77      0.82      0.79       602

    accuracy                           0.81      2218
   macro avg       0.81      0.81      0.81      2218
weighted avg       0.81      0.81      0.81      2218



## Ensemble Logistic Regression & LightGBM (BEST)

In [20]:
from sklearn.ensemble import VotingClassifier

ensemble_clf = VotingClassifier(
    estimators=[
        ('lr', logR),
        ('lgbm', lgbm_clf),
    ],
    voting='soft'
)
ensemble_clf.fit(X_train, y_train_cls)
y_pred_ensemble = ensemble_clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test_cls, y_pred_ensemble))
print(classification_report(y_test_cls, y_pred_ensemble, target_names=emotion_cols))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.237482 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 28074
[LightGBM] [Info] Number of data points in the train set: 8872, number of used features: 4885
[LightGBM] [Info] Start training from score -1.326784
[LightGBM] [Info] Start training from score -1.548442
[LightGBM] [Info] Start training from score -1.439535
[LightGBM] [Info] Start training from score -1.255076
Accuracy: 0.8237150586113616
              precision    recall  f1-score   support

         joy       0.87      0.87      0.87       615
     sadness       0.82      0.79      0.80       475
       anger       0.85      0.79      0.82       526
        fear       0.77      0.82      0.79       602

    accuracy                           0.82      2218
   macro avg       0.83      0.82      0.82      2218
weighted avg       0

/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:1218: UserWarning: Converting data to scipy sparse matrix.
  _log_warning("Converting data to scipy sparse matrix.")


# Final Model Pipeline

In [21]:
def EmoIntPipeline(texts):
    if isinstance(texts, str):
        texts = [texts]

    # Clean text and extract tfidf
    texts = [clean_text(t) for t in texts]
    tfidf_feat = tfidf.transform(texts)

    # Extract lexicons
    lex_feat = np.array([extract_lex(t, nrc_lexicon) for t in texts])
    vad_feat = np.array([extract_vad(t, nrc_vad_lexicon) for t in texts])
    hash_feat = np.array([extract_hash_emo(t, hash_emo_lex) for t in texts])

    # Scale lexicons
    lex_feat = scaler_lex.transform(lex_feat)
    vad_feat = scaler_vad.transform(vad_feat)
    hash_feat = scaler_hash.transform(hash_feat)

    # Combine lexicons
    combined_lex = np.concatenate([vad_feat, lex_feat, hash_feat], axis=1)
    combined_feat = hstack([tfidf_feat, combined_lex])

    # Classification
    probs = ensemble_clf.predict_proba(combined_feat)
    confidence_scores = np.max(probs, axis=1)
    predicted_classes = np.argmax(probs, axis=1)
    pred_emotion = [emotion_cols[i] for i in predicted_classes]

    # Regression
    pred_reg_all = ensemble_reg.predict(combined_feat)
    pred_intensity = [pred[i] for pred, i in zip(pred_reg_all, predicted_classes)]

    rets = []
    for text, em, score, conf in zip(texts, pred_emotion, pred_intensity, confidence_scores):
        if conf > 0.4 and score < 0.4:
            boost = 0.3 * conf
            score = min(score + boost, 1.0)
        rets.append((text, em, score, conf))

    return rets

In [22]:
sample_text = [
    "I'm so happy and grateful today!",
    "I'm kinda happy today, but at the same time life is so bland",
    "I might be happy today, but it's just a normal day.",
    "This makes me so angry, I can't believe they did that.",
    "I'm angry, but I think I can tolerate their behavior.",
    "I'm extremely furious, he never get things right.",
    "I'm feeling a bit down today, things aren't going as planned.",
    "My girlfriend just dumped me, I don't know what to do with my life anymore.",
    "I'm crying my eyes out now, a family member of mine just passed away.",
    "That movie was terrifying, I couldn't sleep all night.",
    "The haunted house was scary, but we had so much fun",
    "That scared me so much, I almost had a heart attack."
]
rets = EmoIntPipeline(sample_text)
for text, emotion, intensity, confidence in rets:
    print(f"Text: {text}")
    print(f"Emotion: {emotion}")
    print(f"Intensity: {intensity:.3f}")
    # print(f"Probability: {confidence:.3f}")
    print("-" * 40)

Text: i am so happy and grateful today!
Emotion: joy
Intensity: 0.884
----------------------------------------
Text: i am kind of happy today, but at the same time life is so bland
Emotion: joy
Intensity: 0.646
----------------------------------------
Text: i might be happy today, but it is just a normal day.
Emotion: joy
Intensity: 0.570
----------------------------------------
Text: this makes me so angry, i cannot believe they did that.
Emotion: anger
Intensity: 0.657
----------------------------------------
Text: i am angry, but i think i can tolerate their behavior.
Emotion: anger
Intensity: 0.460
----------------------------------------
Text: i am extremely furious, he never get things right.
Emotion: anger
Intensity: 0.526
----------------------------------------
Text: i am feeling a bit down today, things are not going as planned.
Emotion: sadness
Intensity: 0.365
----------------------------------------
Text: my girlfriend just dumped me, i do not know what to do with my life 

/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:1218: UserWarning: Converting data to scipy sparse matrix.
  _log_warning("Converting data to scipy sparse matrix.")
